In [1]:
#pip install --upgrade pip


In [2]:
#pip install html5lib

In [3]:
#pip install selenium

In [4]:
#pip install bs4

In [5]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



In [ ]:


driver = webdriver.Chrome()
driver.get("https://www.foxnews.com/category/shows/tucker-carlson-tonight/transcript")
button = driver.find_element(By.XPATH, "//div[@class='button load-more js-load-more']") 
x = 0
for x in range(50):
    button.click()    
    time.sleep(1)
    x+=1
    print(x)
page_source = driver.page_source # save the source to feed to beautiful soup 
print("saved")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [ ]:
soup = BeautifulSoup(page_source)


In [ ]:
url_soup = soup.find_all('a', href=re.compile('/opinion/.*'))

In [ ]:
#url_soup

In [ ]:
url_list = []
prefix = 'https://www.foxnews.com/'
for link in url_soup: 
    suffix = re.findall('\"\/(.*?)\"', str(link))
    if suffix:
        url_list.append(prefix + suffix[0])
    

In [ ]:
url_list[52]

In [ ]:
# verifying for one instance
url = url_list[52]
#url = 'https://www.foxnews.com/opinion/tucker-carlson-faa-travel-pete-buttigieg'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.title.string

# #results = soup.find(id='wrapper')
# #body = soup.find_all('p')
# #text = body[1].text

a_tag = soup.find_all('a')
for tag in a_tag:
    tag.decompose()

copy_tag = soup.find_all('p', class_ = "copyright")
for tag in copy_tag:
    tag.decompose()
    
span_tag = soup.find_all('span')
for tag in span_tag:
    tag.decompose()
    
bold_tag = soup.find_all('strong') 
for tag in bold_tag:
    tag.decompose()
    
italic_tag=soup.find_all('i') 
for tag in italic_tag:
    tag.decompose()

other_tag = soup.find_all('p', class_ ="dek")
for tag in other_tag:
    tag.decompose()
    
other_tag2 = soup.find_all('p', class_ ="subscribed hide")
for tag in other_tag2:
    tag.decompose()
    
other_tag3 = soup.find_all('p', class_ ="success hide") 
for tag in other_tag3:
    tag.decompose()



final = soup.find_all('p')
# text=''
# print(final)
finaltext = ''
for p in final:

    try:
        finaltext = finaltext + p.text
    except:
        continue

finaltext
# #finaltext = ' '.join(text)
# #text
# # #soup.find_all('p')
# #soup.find('time').text

In [ ]:

full_df = None
dfs = []

for n, url in enumerate(url_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    time = soup.find('time').text
    title = soup.title.string

    a_tag = soup.find_all('a')
    for tag in a_tag:
        tag.decompose()

    copy_tag = soup.find_all('p', class_ = "copyright")
    for tag in copy_tag:
        tag.decompose()

    span_tag = soup.find_all('span')
    for tag in span_tag:
        tag.decompose()

    bold_tag = soup.find_all('strong') 
    for tag in bold_tag:
        tag.decompose()

    italic_tag=soup.find_all('i') 
    for tag in italic_tag:
        tag.decompose()

    other_tag = soup.find_all('p', class_ ="dek")
    for tag in other_tag:
        tag.decompose()

    other_tag2 = soup.find_all('p', class_ ="subscribed hide")
    for tag in other_tag2:
        tag.decompose()

    other_tag3 = soup.find_all('p', class_ ="success hide") 
    for tag in other_tag3:
        tag.decompose()


    final = soup.find_all('p')
    # text=''
    # print(final)
    finaltext = ''
    for p in final:

        try:
            finaltext = finaltext + p.text
        except:
            continue
    
    data={'url': [url],'timestamp': [time], 'title': [title], 'text': [finaltext]}
    df = pd.DataFrame(data)
    dfs.append(df)


In [ ]:
full_df

In [ ]:
full_df = pd.concat(dfs).reset_index(drop=True)

In [ ]:
full_df.iloc[19,2]

In [ ]:
no_duplicates=full_df.drop_duplicates().reset_index(drop=True)

In [ ]:
len(no_duplicates)

In [ ]:
no_duplicates

In [ ]:
no_duplicates.to_csv('Tucker_transcripts_2.5.23.tsv', sep="\t")

In [ ]:
# Importing modules

import os
os.chdir('..')
# Read data into papers
papers = no_duplicates.reset_index().drop(['index'], axis=1) #.drop([18,20,21], axis=0).reset_index()
# Print head
papers.tail()

In [ ]:
# Load the regular expression library
import re
# Remove punctuation
papers['text_processed'] = \
papers['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['text_processed'] = \
papers['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['text_processed'].head()

In [ ]:
#pip install WordCloud

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
#long_string = ','.join(list(papers['text_processed'].values))
l = []
for index in papers.index:
    #print(index)
    
    long_string = papers['text_processed'][index]

    # Create a WordCloud object
    wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
    # Generate a word cloud
    wordcloud.generate(long_string)
    # add wordcloud image to 'l' list    
    l.append(wordcloud.to_image())
    

In [ ]:
#make new column in papers df off of 'l' list 
papers['img'] = l

In [ ]:
#if you index any image cell it will display the image

papers['img'][9]


In [ ]:
#pip install gensim

In [ ]:
#pip install nltk

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [ ]:
nltk.download('wordnet')

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'tucker', 'carlson', 'fox', 'news', 'channel', 'host', 'evening', 'welcome', 'tonight'])

stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and token not in stop_words and len(token) > 3:

            result.append(lemmatize_stemming(token))
            
    return result

In [ ]:
'''
Preview a document after preprocessing
'''
document_num = 50
doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

In [ ]:
ep1 =  papers['text_processed'][0]
ep1_preprocessed = preprocess(ep1)
#ep1_preprocessed

In [ ]:
processed_docs = []

for transcript in papers.text_processed:
    processed_docs.append(preprocess(transcript))

In [ ]:
'''
Preview 'processed_docs'
'''
#print(processed_docs[:2])



In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
dictionary

In [ ]:

'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    # if count > 10:
    #     break

In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 1
bow_doc_x = bow_corpus[document_num]

# for i in range(len(bow_doc_x)):
#     print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
#                                                      dictionary[bow_doc_x[i][0]], 
#                                                      bow_doc_x[i][1]))

In [ ]:
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 15, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'tucker', 'carlson', 'fox', 'news', 'channel', 'host', 'evening', 'welcome', 'tonight'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]


data = papers.text_processed.values.tolist()
print(data)
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

In [ ]:
#pip install scikit-learn

In [ ]:
#pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
import pickle 

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(doc_lda, corpus, id2word)